# Data collection
This notebook collects the data from different data sources and end points and saves the raw data in a SQLite database.

In the following I search for articles published in a certain time frame in the TowardsDataScience archive (https://towardsdatascience.com/archive/year/month/day) 

References: 
https://hackernoon.com/how-to-scrape-a-medium-publication-a-python-tutorial-for-beginners-o8u3t69

In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

stories_data = []

In [32]:
from datetime import date, timedelta

days = []

sdate = date(2020, 1, 1)   # start date
edate = date(2020, 12, 31)   # end date

delta = edate - sdate       # as timedelta

for i in range(delta.days + 1):
    day = sdate + timedelta(days=i)
    days.append(day)

In [36]:
year = days[k].year
month = days[k].month
day = days[k].days

print()

2020

In [63]:
for k in range(0,1,1):
    year = str(days[k].year)
    month = str(days[k].month).zfill(2)
    day = str(days[k].day).zfill(2)
    
    date = f'{month}/{day}/{year}'
    url = f'https://towardsdatascience.com/archive/{year}/{month}/{day}'
    print("Url: " + url)

#     page = requests.get(url)
#     soup = BeautifulSoup(page.text, 'html.parser')

Url: https://towardsdatascience.com/archive/2020/01/01


## How to get number of users clapped for a story
Since the this data is only visible if you click on the number of claps, I am using selenium to scrape the story HTML together with the section to more detailed information about the claps

In [13]:
from selenium import webdriver

DRIVER_PATH = r".\chrome_driver\chromedriver.exe"
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
#driver.get('towardsdatascience.com/how-to-be-fancy-with-python-8e4c53f4778')
driver.get("https://www.nintendo.com/") 
print(driver.page_source)
driver.quit()



WebDriverException: Message: unknown error: Runtime.executionContextCreated has invalid 'context': {"auxData":{"frameId":"37AA9A3ED47932FDDA15D79E7069E945","isDefault":true,"type":"default"},"id":1,"name":"","origin":"://","uniqueId":"-5668936229025524128.-472603704465957818"}
  (Session info: chrome=91.0.4472.124)
  (Driver info: chromedriver=2.9.248315,platform=Windows NT 6.3 x86_64)


In [85]:
# scrape stories
url_story = "https://towardsdatascience.com/how-to-be-fancy-with-python-8e4c53f47789"
    
page = requests.get(url_story)
soup = BeautifulSoup(page.text, 'html.parser')

soup.find_all('a'):
    if button.get_text() == 'Add to watch list':
        print(button.get('href'))

In [ ]:
search_url=“https://www.google.com/search?q={q}&tbm=isch&tbs=sur%3Afc&hl=en&ved=0CAIQpwVqFwoTCKCa1c6s4-oCFQAAAAAdAAAAABAC&biw=1251&bih=568" 

driver.get(search_url.format(q='Car'))

In [66]:
def extract_data(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    stories = soup.find_all('div', class_='streamItem streamItem--postPreview js-streamItem')
    stories_data = []
    
    # find attributes author_url, reading_time, reading_time, responses, story_url for the stories which where published 
    # on the specified date
    
    for story in stories:
        print(story)
        each_story = []

        author_box = story.find('div', class_='postMetaInline u-floatLeft u-sm-maxWidthFullWidth')
        author_url = author_box.find('a')['href']

        try:
            reading_time = author_box.find('span', class_='readingTime')['title']
        except:
            continue

        title = story.find('h3').text if story.find('h3') else '-'
        subtitle = story.find('h4').text if story.find('h4') else '-'

        if story.find('button', class_='button button--chromeless u-baseColor--buttonNormal js-multirecommendCountButton u-disablePointerEvents'):

            claps = story.find('button', class_='button button--chromeless u-baseColor--buttonNormal js-multirecommendCountButton u-disablePointerEvents').text

        else:
            claps = 0

        if story.find('a', class_='button button--chromeless u-baseColor--buttonNormal'):

            responses = story.find('a', class_='button button--chromeless u-baseColor--buttonNormal').text

        else:
            responses = '0 responses'

        story_url = story.find('a', class_='button button--smaller button--chromeless u-baseColor--buttonNormal')['href']
   
        # data cleaning
        reading_time = reading_time.split()[0]
        responses = responses.split()[0]

        story_page = requests.get(story_url)
        story_soup = BeautifulSoup(story_page.text, 'html.parser')

        sections = story_soup.find_all('section')
        story_paragraphs = []
        section_titles = []

        for section in sections:
            paragraphs = section.find_all('p')
            for paragraph in paragraphs:
                story_paragraphs.append(paragraph.text)

            subs = section.find_all('h1')
            for sub in subs:
                section_titles.append(sub.text)

        number_sections = len(section_titles)
        number_paragraphs = len(story_paragraphs)

        each_story.append(date)
        each_story.append(title)
        each_story.append(subtitle)
        each_story.append(claps)
        each_story.append(responses)
        each_story.append(author_url)
        each_story.append(story_url)
        each_story.append(reading_time)
        each_story.append(number_sections)
        each_story.append(section_titles)
        each_story.append(number_paragraphs)
        each_story.append(story_paragraphs)

        stories_data.append(each_story)

    # write data to data frame "df" and return df
    columns = ['date', 'title', 'subtitle', 'claps', 'responses', 
           'author_url', 'story_url', 'reading_time (mins)', 
           'number_sections', 'section_titles', 
           'number_paragraphs', 'paragraphs']

    df = pd.DataFrame(stories_data, columns=columns)
    return df

df = extract_data(url)
df

<div class="streamItem streamItem--postPreview js-streamItem"><div class="cardChromeless u-marginTop20 u-paddingTop10 u-paddingBottom15 u-paddingLeft20 u-paddingRight20"><div class="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls" data-post-id="c1cd79bd1b32" data-source="collection_archive---------0-----------------------"><div class="u-clearfix u-marginBottom15 u-paddingTop5"><div class="postMetaInline u-floatLeft u-sm-maxWidthFullWidth"><div class="u-flexCenter"><div class="postMetaInline-avatar u-flex0"><a class="link u-baseColor--link avatar" data-action="show-user-card" data-action-type="hover" data-action-value="4cbd6b36e62a" data-collection-slug="towards-data-science" data-user-id="4cbd6b36e62a" dir="auto" href="https://towardsdatascience.com/@martin.heinz"><img alt="Go to the profile of Martin Heinz" class="avatar-image u-size36x36 u-xs-size32x32" src="https://cdn-images-1.medium.com/fit/c/72/72/2*p3sIQ1Ga2bfAbZYzoCcACw.jpeg"/></a></di

<div class="streamItem streamItem--postPreview js-streamItem"><div class="cardChromeless u-marginTop20 u-paddingTop10 u-paddingBottom15 u-paddingLeft20 u-paddingRight20"><div class="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls" data-post-id="3c46fb61de3b" data-source="collection_archive---------1-----------------------"><div class="u-clearfix u-marginBottom15 u-paddingTop5"><div class="postMetaInline u-floatLeft u-sm-maxWidthFullWidth"><div class="u-flexCenter"><div class="postMetaInline-avatar u-flex0"><a class="link u-baseColor--link avatar" data-action="show-user-card" data-action-type="hover" data-action-value="52b8b739a77b" data-collection-slug="towards-data-science" data-user-id="52b8b739a77b" dir="auto" href="https://towardsdatascience.com/@himanshurawlani"><img alt="Go to the profile of Himanshu Rawlani" class="avatar-image u-size36x36 u-xs-size32x32" src="https://cdn-images-1.medium.com/fit/c/72/72/1*PW-GmQGhWGE7V9sOOpASEA.jpeg"/><

<div class="streamItem streamItem--postPreview js-streamItem"><div class="cardChromeless u-marginTop20 u-paddingTop10 u-paddingBottom15 u-paddingLeft20 u-paddingRight20"><div class="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls" data-post-id="dc2e7b804acc" data-source="collection_archive---------2-----------------------"><div class="u-clearfix u-marginBottom15 u-paddingTop5"><div class="postMetaInline u-floatLeft u-sm-maxWidthFullWidth"><div class="u-flexCenter"><div class="postMetaInline-avatar u-flex0"><a class="link u-baseColor--link avatar" data-action="show-user-card" data-action-type="hover" data-action-value="9a831eeffa24" data-collection-slug="towards-data-science" data-user-id="9a831eeffa24" dir="auto" href="https://towardsdatascience.com/@edenau"><img alt="Go to the profile of Eden Au" class="avatar-image u-size36x36 u-xs-size32x32" src="https://cdn-images-1.medium.com/fit/c/72/72/1*aWG_hsKc2xa-lP6H1qhAwA.jpeg"/></a></div><div clas

<div class="streamItem streamItem--postPreview js-streamItem"><div class="cardChromeless u-marginTop20 u-paddingTop10 u-paddingBottom15 u-paddingLeft20 u-paddingRight20"><div class="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls" data-post-id="8e4c53f47789" data-source="collection_archive---------3-----------------------"><div class="u-clearfix u-marginBottom15 u-paddingTop5"><div class="postMetaInline u-floatLeft u-sm-maxWidthFullWidth"><div class="u-flexCenter"><div class="postMetaInline-avatar u-flex0"><a class="link u-baseColor--link avatar" data-action="show-user-card" data-action-type="hover" data-action-value="db5934347fa" data-collection-slug="towards-data-science" data-user-id="db5934347fa" dir="auto" href="https://towardsdatascience.com/@dipam44"><img alt="Go to the profile of Dipam Vasani" class="avatar-image u-size36x36 u-xs-size32x32" src="https://cdn-images-1.medium.com/fit/c/72/72/1*OM71SRu62-jO0qb7E20t-w.png"/></a></div><div c

<div class="streamItem streamItem--postPreview js-streamItem"><div class="cardChromeless u-marginTop20 u-paddingTop10 u-paddingBottom15 u-paddingLeft20 u-paddingRight20"><div class="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls" data-post-id="c14698f322f5" data-source="collection_archive---------4-----------------------"><div class="u-clearfix u-marginBottom15 u-paddingTop5"><div class="postMetaInline u-floatLeft u-sm-maxWidthFullWidth"><div class="u-flexCenter"><div class="postMetaInline-avatar u-flex0"><a class="link u-baseColor--link avatar" data-action="show-user-card" data-action-type="hover" data-action-value="afe93926edf5" data-collection-slug="towards-data-science" data-user-id="afe93926edf5" dir="auto" href="https://towardsdatascience.com/@patriciathaine"><div class="u-relative u-inlineBlock u-flex0"><img alt="Go to the profile of Patricia Thaine" class="avatar-image u-size36x36 u-xs-size32x32" src="https://cdn-images-1.medium.com/f

...

<div class="streamItem streamItem--postPreview js-streamItem"><div class="cardChromeless u-marginTop20 u-paddingTop10 u-paddingBottom15 u-paddingLeft20 u-paddingRight20"><div class="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls" data-post-id="7e33d6ff0a5c" data-source="collection_archive---------27-----------------------"><div class="u-clearfix u-marginBottom15 u-paddingTop5"><div class="postMetaInline u-floatLeft u-sm-maxWidthFullWidth"><div class="u-flexCenter"><div class="postMetaInline-avatar u-flex0"><a class="link u-baseColor--link avatar" data-action="show-user-card" data-action-type="hover" data-action-value="62bfe8130237" data-collection-slug="towards-data-science" data-user-id="62bfe8130237" dir="auto" href="https://towardsdatascience.com/@jakebatsuuri"><div class="u-relative u-inlineBlock u-flex0"><img alt="Go to the profile of Jake Batsuuri" class="avatar-image u-size36x36 u-xs-size32x32" src="https://cdn-images-1.medium.com/fit/

<div class="streamItem streamItem--postPreview js-streamItem"><div class="cardChromeless u-marginTop20 u-paddingTop10 u-paddingBottom15 u-paddingLeft20 u-paddingRight20"><div class="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls" data-post-id="a61ba00101e9" data-source="collection_archive---------28-----------------------"><div class="u-clearfix u-marginBottom15 u-paddingTop5"><div class="postMetaInline u-floatLeft u-sm-maxWidthFullWidth"><div class="u-flexCenter"><div class="postMetaInline-avatar u-flex0"><a class="link u-baseColor--link avatar" data-action="show-user-card" data-action-type="hover" data-action-value="2c1f793c1f32" data-collection-slug="towards-data-science" data-user-id="2c1f793c1f32" dir="auto" href="https://towardsdatascience.com/@snehas_9179"><img alt="Go to the profile of Sneha Somaya" class="avatar-image u-size36x36 u-xs-size32x32" src="https://cdn-images-1.medium.com/fit/c/72/72/2*PSBoXYFV4ER8PaVpuscM0Q.jpeg"/></a></di

<div class="streamItem streamItem--postPreview js-streamItem"><div class="cardChromeless u-marginTop20 u-paddingTop10 u-paddingBottom15 u-paddingLeft20 u-paddingRight20"><div class="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls" data-post-id="145c5daefcc5" data-source="collection_archive---------29-----------------------"><div class="u-clearfix u-marginBottom15 u-paddingTop5"><div class="postMetaInline u-floatLeft u-sm-maxWidthFullWidth"><div class="u-flexCenter"><div class="postMetaInline-avatar u-flex0"><a class="link u-baseColor--link avatar" data-action="show-user-card" data-action-type="hover" data-action-value="937d2ce2cfe6" data-collection-slug="towards-data-science" data-user-id="937d2ce2cfe6" dir="auto" href="https://towardsdatascience.com/@danielwilms"><img alt="Go to the profile of Daniel Wilms" class="avatar-image u-size36x36 u-xs-size32x32" src="https://cdn-images-1.medium.com/fit/c/72/72/2*bo4DFv6Cz52WJTlHgbTVQg.jpeg"/></a></di

<div class="streamItem streamItem--postPreview js-streamItem"><div class="cardChromeless u-marginTop20 u-paddingTop10 u-paddingBottom15 u-paddingLeft20 u-paddingRight20"><div class="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls" data-post-id="253fa0124629" data-source="collection_archive---------30-----------------------"><div class="u-clearfix u-marginBottom15 u-paddingTop5"><div class="postMetaInline u-floatLeft u-sm-maxWidthFullWidth"><div class="u-flexCenter"><div class="postMetaInline-avatar u-flex0"><a class="link u-baseColor--link avatar" data-action="show-user-card" data-action-type="hover" data-action-value="231e5275c3b6" data-collection-slug="towards-data-science" data-user-id="231e5275c3b6" dir="auto" href="https://towardsdatascience.com/@dglid"><img alt="Go to the profile of David Glidden" class="avatar-image u-size36x36 u-xs-size32x32" src="https://cdn-images-1.medium.com/fit/c/72/72/1*4okL6g5qPRhWHu_1wLP5cA.jpeg"/></a></div><di

,date,title,subtitle,claps,responses,author_url,story_url,reading_time (mins),number_sections,section_titles,number_paragraphs,paragraphs
0,01/01/2020,Making Python Programs Blazingly Fast,Let’s look at the performance of our Python pr...,3.3K,3,https://towardsdatascience.com/@martin.heinz,https://towardsdatascience.com/making-python-p...,5,4,"[Making Python Programs Blazingly Fast, Timing...",27,"[Martin Heinz, Jan 1, 2020·5 min read, Python ..."
1,01/01/2020,Implementing a fully convolutional network (FC...,"A tutorial on building, training and…",403,3,https://towardsdatascience.com/@himanshurawlani,https://towardsdatascience.com/implementing-a-...,11,8,[Understanding and implementing a fully convol...,36,"[Himanshu Rawlani, Jan 1, 2020·11 min read, Co..."
2,01/01/2020,6 New Features in Python 3.8 for Python Newbies,Python Beginner,1.8K,4,https://towardsdatascience.com/@edenau,https://towardsdatascience.com/6-new-features-...,4,5,[6 New Features in Python 3.8 for Python Newbi...,26,"[Eden Au, Jan 1, 2020·4 min read, Languages ch..."
3,01/01/2020,How to be fancy with Python,Python tricks that will make your life easier,1.7K,12,https://towardsdatascience.com/@dipam44,https://towardsdatascience.com/how-to-be-fancy...,5,1,[How to be fancy with Python],30,"[Dipam Vasani, Jan 1, 2020·5 min read, Python ..."
4,01/01/2020,Perfectly Privacy-Preserving AI,Data Privacy,331,2,https://towardsdatascience.com/@patriciathaine,https://towardsdatascience.com/perfectly-priva...,10,9,"[Perfectly Privacy-Preserving AI, The Four Pil...",45,"[Patricia Thaine, Jan 1, 2020·10 min read, Dat..."
5,01/01/2020,From scratch to search: playing with your data...,One Pipeline to rule…,231,1,https://towardsdatascience.com/@stanislavprihoda,https://towardsdatascience.com/from-scratch-to...,9,11,[From scratch to search: playing with your dat...,47,"[Stanislav Prihoda 🔥, Jan 1, 2020·9 min read, ..."
6,01/01/2020,GAN Pix2Pix Generative Model,Image-to-image translation with Pix2Pix model,87,1,https://towardsdatascience.com/@baakchsu.sprx77,https://towardsdatascience.com/gan-pix2pix-gen...,6,9,"[GAN Pix2Pix Generative Model, Pix2Pix GAN: In...",22,"[Anirudh S, Jan 1, 2020·6 min read, We hear a ..."
7,01/01/2020,An Introduction to Decision Trees with Python ...,A complete guide to getting an intuitive under...,105,1,https://towardsdatascience.com/@mikkelduif,https://towardsdatascience.com/an-introduction...,8,5,[An Introduction to Decision Trees with Python...,19,"[Mikkel Duif, Jan 1, 2020·8 min read, Decision..."
8,01/01/2020,Decision Trees for Dummies,An Intuitive Approach,84,1,https://towardsdatascience.com/@pratishgupta91,https://towardsdatascience.com/decision-trees-...,5,1,[Decision Trees for Dummies],21,"[Pratish, Jan 1, 2020·5 min read, Let’s look a..."
9,01/01/2020,Kaggle User Survey 2019,"A dashboard made using R, Flexdashboard, and H...",73,0,https://towardsdatascience.com/@theairbend3r,https://towardsdatascience.com/kaggle-user-sur...,6,4,"[Kaggle User Survey Dashboard— 2019, Home, Wom...",23,"[Akshaj Verma, Jan 1, 2020·6 min read, A dashb..."
